Import the seillra package, and other helpful packages:

In [ ]:
import os, sys
import pandas as pd
import torch
import numpy as np
import collections

import seillra as sl
import seimodel as sm

To load different parts of the models, select a rank, whether the output is chromatin profiles or sequence classes, whether you are predicting for any sequences or specifically variants, and if the model should be on the CPU or GPU. These are selectable when using the `Sei_LLRA` model class, but other functionallity can be customized using the model blocks explicitly.

In [ ]:
# If using a MAC
if "qnnpack" in torch.backends.quantized.supported_engines:
    torch.backends.quantized.engine = "qnnpack"

In [ ]:
rank = 256 # 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048
sequence_classes = False # True
sequence_type = "sequence" # "variant"
device = "cpu" # "cuda", "cuda:0"
model = sl.Sei_LLRA(k=rank, projection=sequence_classes, mode = sequence_type, device=device)

This can be done using `torch.nn.Sequential` as well. For an un-quantized (GPU) model see the commented code. Note that one should be careful about handeling forward and reverse-complement sequences. 

In [ ]:
mod_trunk = sl.get_sei_trunk_q() # sm.get_sei_trunk().load_weights()
mod_head  = sl.get_sei_head_llra_q(k=rank) # sl.get_sei_head_llra(k=rank)
mod_projection = sm.get_sei_projection().load_weights()
mod_projection.set_mode(sequence_type)

#- Make a full model
mod = torch.nn.Sequential(collections.OrderedDict([
    ('trunk', mod_trunk),
    ('head', mod_head),
    ('projection', mod_projection)
]))

Here is an example of running the model on a random one-hot encoded sequence.

In [ ]:
sequences = torch.randint(0, 4, (16, 4096))
x = torch.nn.functional.one_hot(sequences, num_classes=4).permute(0, 2, 1).float()
print(x.shape)
# - run the model
out = model(x.to("cpu"))
print(out.shape)

Now we can try a different rank, use the sequenc classes and use the GPU. Note that this will remain on the CPU if you do not have access to a cuda enabled GPU (Apple M-series GPUs are not cuda enabled).

In [ ]:
rank = 1 # 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048
sequence_classes = True # True
sequence_type = "sequence" # "variant"
device = "cuda" # "cpu", "cuda:0"
model2 = sl.Sei_LLRA(k=rank, projection=sequence_classes, mode = sequence_type, device=device)


In [ ]:
sequences = torch.randint(0, 4, (16, 4096))
x = torch.nn.functional.one_hot(sequences, num_classes=4).permute(0, 2, 1).float()
print(x.shape)
# - run the model
out = model2(x.to("cuda"))
print(out.shape)

This can also be done for getting predictions for variants. We will set up example reference and alternate allele sequences. 

In [ ]:
rank = 1 # 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048
sequence_classes = True # False
sequence_type = "variant" # "sequence"
device = "cpu" # "cuda", "cuda:0"
model3 = sl.Sei_LLRA(k=rank, projection=sequence_classes, mode = sequence_type, device=device)


In [ ]:
ref_sequences = torch.randint(0, 4, (16, 4096))
alt_sequences = ref_sequences.clone()
center_idx = 4096 // 2
alt_sequences[:, center_idx] = (ref_sequences[:, center_idx] + 1) % 4


x_ref = torch.nn.functional.one_hot(ref_sequences, num_classes=4).permute(0, 2, 1).float()
x_alt = torch.nn.functional.one_hot(alt_sequences, num_classes=4).permute(0, 2, 1).float()

print(x_ref.shape, x_alt.shape)
input = (x_ref, x_alt)
out_ref, out_alt = model3(input)
print(out_ref.shape, out_alt.shape)


Here is an example workflow for doing variant effect predictions. For instructions on downloading data, see manuscript repository: [https://github.com/egilfeather/lowrank-s2f-code](https://github.com/egilfeather/lowrank-s2f-code).

In [ ]:

file_path = "./data/MPRA_eQTL.vcf"
df = pd.read_csv("./data/MPRA_eQTL.tsv", sep='\t', header=0)
print(df.head())

In [ ]:
import seillra as sl
rank = 256
model = sl.Sei_LLRA(k=rank, projection=True, mode = "variant", device="cpu")

In [ ]:
from tqdm import tqdm
from sei_lora.dataloaders import VariantDataset, VariantDataLoader
dataset = VariantDataset(file_path=file_path, fasta_path = "./resources/hg38_UCSC.fa", window_size = 4096)
dataloader = VariantDataLoader(dataset=dataset, batch_size=32, shuffle=False, num_workers=0)
model.eval()

all_cp_ref = []
all_cp_alt = []
all_vcf = []

progress_bar = tqdm(dataloader, desc=f"Running {rank} benchmark")

for batch in progress_bar:
    ref, alt, vcf = batch


    cp_outputs = model((ref, alt))  # both are tuples: (refproj, altproj)


    all_cp_ref.append(cp_outputs[0])
    all_cp_alt.append(cp_outputs[1])
    all_vcf.append(vcf)

    # Accumulate by appending to list

all_cp_ref = torch.cat([t.detach() for t in all_cp_ref], dim=0).numpy()
all_cp_alt = torch.cat([t.detach() for t in all_cp_alt], dim=0).numpy()

all_vcf = np.concatenate(all_vcf, axis=0)



In [ ]:
from sklearn.metrics import roc_auc_score
sc_diff = all_cp_alt - all_cp_ref

df_pred = pd.DataFrame(all_vcf, columns=["CHROM", "POS", "NAME", "REF", "ALT"])
df_pred["POS"] = df_pred["POS"].astype(int)

seqclass_path = os.path.join( "./resources/seqclass.names")
with open(seqclass_path, "r") as f:
    sc_names = []
    for line in f:
        parts = line.strip().split()
        if len(parts) > 1:
            sc_names.append("-".join(parts[1:]))
        else:
            sc_names.append(parts[0])

df_sc = pd.DataFrame(sc_diff[:, :40], columns=sc_names[:40])

df_pred =  pd.concat([df_pred, df_sc], axis=1)


df_ou = df[df['consequence'].isin(['over', 'under'])].copy()
df_combine_ou = df_ou.merge(df_pred, left_on = ["chrom", "pos", "ref", "alt"], right_on=["CHROM", "POS", "REF", "ALT"], how = "inner")
binary_labels_ou = (df_combine_ou['consequence'] == 'over')
roc_promoter_ou = roc_auc_score(binary_labels_ou, df_combine_ou["Promoter"])
print(roc_promoter_ou)
